In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Dropout, Conv2D, BatchNormalization, Flatten, Dense, MaxPool2D, Input, Activation, Add, MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, Adagrad, Adadelta, Adamax, RMSprop
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
fldr = "jangedoo/utkface-new/versions/1/UTKFace"

In [3]:
files = os.listdir(fldr)

In [4]:
ages = []
genders = []
images = []

In [5]:
for f in files:
    age = int(f.split('_')[0])
    ages.append(age)
    gender = int(f.split('_')[1])
    genders.append(gender)
    image = cv2.imread(fldr + '/' + f)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (48, 48))
    images.append(image)

In [6]:
images_f  = np.array(images)
ages_f = np.array(ages)
genders_f = np.array(genders)

In [7]:
# Defining labels for the model

labels = []
i = 0
while i < len(ages_f):
    label = []
    label.append(ages_f[i])
    label.append(genders_f[i])
    labels.append(label)
    i += 1

In [8]:
labels_f = np.array(labels)

In [9]:
# Creating another list of images with the pixel values scaled between 0 and 1

images_f_2 = images_f / 255.0

In [10]:
X_train, X_test, y_train, y_test = train_test_split(images_f_2, labels_f, test_size=0.25, random_state=42)

In [11]:
y_train_2=[y_train[:,1],y_train[:,0]]
y_test_2=[y_test[:,1],y_test[:,0]]

In [12]:
# using Tensorflow

def Convolution(input_tensor, filters):
    x = Conv2D(filters=filters, kernel_size=(3,3), padding="same", strides=(1,1), kernel_regularizer=l2(0.001))(input_tensor)
    x = Dropout(0.1)(x)
    x = Activation('relu')(x)
    return x

In [13]:
def model(input_shape):
    inputs = Input((input_shape))
    conv_1 = Convolution(inputs, 32)
    maxp_1 = MaxPooling2D(pool_size=(2,2))(conv_1)
    conv_2 = Convolution(maxp_1, 64)
    maxp_2 = MaxPooling2D(pool_size=(2,2))(conv_2)
    conv_3 = Convolution(maxp_2, 128)
    maxp_3 = MaxPooling2D(pool_size=(2,2))(conv_3)
    conv_4 = Convolution(maxp_3, 256)
    maxp_4 = MaxPooling2D(pool_size=(2,2))(conv_4)
    flatten = Flatten()(maxp_4)
    dense_1 = Dense(64, activation='relu')(flatten)
    dense_2 = Dense(64, activation='relu')(flatten)
    drop_1 = Dropout(0.2)(dense_1)
    drop_2 = Dropout(0.2)(dense_2)
    output_1 = Dense(1, activation='sigmoid', name='gender_out')(drop_1)
    output_2 = Dense(1, activation='relu', name='age_out')(drop_2)
    model = Model(inputs=inputs, outputs=[output_1, output_2])
    model.compile(optimizer='adam', loss=['binary_crossentropy', 'mae'], metrics=['accuracy', 'accuracy'])
    return model

In [14]:
model = model((48, 48, 3))

In [15]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 48, 48, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 48, 48,    │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 48, 48,    │          0 │ conv2d[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 48, 48,    │          0 │ dropout[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 24, 24,    │          0 │ activation[0][0]  │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 24, 24,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 24, 24,    │          0 │ conv2d_1[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 24, 24,    │          0 │ dropout_1[0][0]   │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 12, 12,    │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 12, 12,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 12, 12,    │          0 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 12, 12,    │          0 │ dropout_2[0][0]   │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 6, 6, 128) │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 6, 6, 256) │    295,168 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 6, 6, 256) │          0 │ conv2d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 6, 6, 256) │          0 │ dropout_3[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 3, 3, 256) │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 683,586 (2.61 MB)

 Trainable params: 683,586 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
fle_s = 'Age_Sex_Detection.h5'
checkpoint = ModelCheckpoint(fle_s, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')
early_stop = EarlyStopping(monitor='val_loss', patience=75, restore_best_weights=True)
callback_list = [checkpoint, early_stop]

In [17]:
history = model.fit(X_train, y_train_2, batch_size=64, validation_data=(X_test,y_test_2), epochs=250, callbacks=callback_list)

Epoch 1/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - age_out_accuracy: 0.0469 - age_out_loss: 17.3227 - gender_out_accuracy: 0.5245 - gender_out_loss: 0.7066 - loss: 18.2477
Epoch 1: val_loss improved from inf to 16.00834, saving model to Age_Sex_Detection.h5


278/278 ━━━━━━━━━━━━━━━━━━━━ 23s 79ms/step - age_out_accuracy: 0.0469 - age_out_loss: 17.3176 - gender_out_accuracy: 0.5245 - gender_out_loss: 0.7065 - loss: 18.2424 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 15.1386 - val_gender_out_accuracy: 0.4903 - val_gender_out_loss: 0.6931 - val_loss: 16.0083
Epoch 2/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - age_out_accuracy: 0.0499 - age_out_loss: 14.2344 - gender_out_accuracy: 0.5152 - gender_out_loss: 0.6864 - loss: 15.1015
Epoch 2: val_loss improved from 16.00834 to 14.57126, saving model to Age_Sex_Detection.h5


278/278 ━━━━━━━━━━━━━━━━━━━━ 24s 86ms/step - age_out_accuracy: 0.0499 - age_out_loss: 14.2317 - gender_out_accuracy: 0.5154 - gender_out_loss: 0.6863 - loss: 15.0988 - val_age_out_accuracy: 0.0471 - val_age_out_loss: 13.7003 - val_gender_out_accuracy: 0.6774 - val_gender_out_loss: 0.6162 - val_loss: 14.5713
Epoch 3/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - age_out_accuracy: 0.0447 - age_out_loss: 10.9691 - gender_out_accuracy: 0.6844 - gender_out_loss: 0.6029 - loss: 11.8325
Epoch 3: val_loss improved from 14.57126 to 12.32485, saving model to Age_Sex_Detection.h5


278/278 ━━━━━━━━━━━━━━━━━━━━ 23s 82ms/step - age_out_accuracy: 0.0447 - age_out_loss: 10.9681 - gender_out_accuracy: 0.6845 - gender_out_loss: 0.6028 - loss: 11.8316 - val_age_out_accuracy: 0.0466 - val_age_out_loss: 11.4799 - val_gender_out_accuracy: 0.7459 - val_gender_out_loss: 0.5400 - val_loss: 12.3248
Epoch 4/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - age_out_accuracy: 0.0448 - age_out_loss: 10.0493 - gender_out_accuracy: 0.7487 - gender_out_loss: 0.5152 - loss: 10.8692
Epoch 4: val_loss did not improve from 12.32485
278/278 ━━━━━━━━━━━━━━━━━━━━ 24s 87ms/step - age_out_accuracy: 0.0448 - age_out_loss: 10.0483 - gender_out_accuracy: 0.7487 - gender_out_loss: 0.5151 - loss: 10.8682 - val_age_out_accuracy: 0.0456 - val_age_out_loss: 11.6821 - val_gender_out_accuracy: 0.7773 - val_gender_out_loss: 0.4972 - val_loss: 12.5147
Epoch 5/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - age_out_accuracy: 0.0431 - age_out_loss: 9.1815 - gender_out_accuracy: 0.7658 - gender_out_loss: 0.48

278/278 ━━━━━━━━━━━━━━━━━━━━ 24s 88ms/step - age_out_accuracy: 0.0431 - age_out_loss: 9.1813 - gender_out_accuracy: 0.7659 - gender_out_loss: 0.4834 - loss: 9.9969 - val_age_out_accuracy: 0.0466 - val_age_out_loss: 11.0698 - val_gender_out_accuracy: 0.8085 - val_gender_out_loss: 0.4340 - val_loss: 11.8591
Epoch 6/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - age_out_accuracy: 0.0472 - age_out_loss: 8.7832 - gender_out_accuracy: 0.7855 - gender_out_loss: 0.4540 - loss: 9.5915
Epoch 6: val_loss improved from 11.85905 to 11.25318, saving model to Age_Sex_Detection.h5


...

278/278 ━━━━━━━━━━━━━━━━━━━━ 29s 104ms/step - age_out_accuracy: 0.0509 - age_out_loss: 5.7977 - gender_out_accuracy: 0.8918 - gender_out_loss: 0.2491 - loss: 6.7058 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 7.7870 - val_gender_out_accuracy: 0.8937 - val_gender_out_loss: 0.2488 - val_loss: 8.7081
Epoch 26/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - age_out_accuracy: 0.0477 - age_out_loss: 5.8421 - gender_out_accuracy: 0.8879 - gender_out_loss: 0.2504 - loss: 6.7658
Epoch 26: val_loss improved from 8.70815 to 8.59108, saving model to Age_Sex_Detection.h5


278/278 ━━━━━━━━━━━━━━━━━━━━ 30s 107ms/step - age_out_accuracy: 0.0477 - age_out_loss: 5.8425 - gender_out_accuracy: 0.8879 - gender_out_loss: 0.2504 - loss: 6.7662 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 7.6185 - val_gender_out_accuracy: 0.8711 - val_gender_out_loss: 0.2852 - val_loss: 8.5911
Epoch 27/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - age_out_accuracy: 0.0485 - age_out_loss: 5.6282 - gender_out_accuracy: 0.8909 - gender_out_loss: 0.2407 - loss: 6.5548
Epoch 27: val_loss did not improve from 8.59108
278/278 ━━━━━━━━━━━━━━━━━━━━ 29s 106ms/step - age_out_accuracy: 0.0485 - age_out_loss: 5.6286 - gender_out_accuracy: 0.8910 - gender_out_loss: 0.2408 - loss: 6.5553 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 8.1408 - val_gender_out_accuracy: 0.8829 - val_gender_out_loss: 0.2538 - val_loss: 9.0943
Epoch 28/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - age_out_accuracy: 0.0466 - age_out_loss: 5.6220 - gender_out_accuracy: 0.8951 - gender_out_loss: 0.2452 - 

278/278 ━━━━━━━━━━━━━━━━━━━━ 30s 106ms/step - age_out_accuracy: 0.0477 - age_out_loss: 5.5938 - gender_out_accuracy: 0.8935 - gender_out_loss: 0.2434 - loss: 6.5493 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 6.9140 - val_gender_out_accuracy: 0.8984 - val_gender_out_loss: 0.2446 - val_loss: 7.8782
Epoch 30/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - age_out_accuracy: 0.0460 - age_out_loss: 5.5998 - gender_out_accuracy: 0.8961 - gender_out_loss: 0.2347 - loss: 6.5581
Epoch 30: val_loss did not improve from 7.87820
278/278 ━━━━━━━━━━━━━━━━━━━━ 31s 110ms/step - age_out_accuracy: 0.0460 - age_out_loss: 5.5998 - gender_out_accuracy: 0.8960 - gender_out_loss: 0.2347 - loss: 6.5581 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 8.3594 - val_gender_out_accuracy: 0.8969 - val_gender_out_loss: 0.2440 - val_loss: 9.3407
Epoch 31/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - age_out_accuracy: 0.0466 - age_out_loss: 5.5633 - gender_out_accuracy: 0.9014 - gender_out_loss: 0.2348 - 

278/278 ━━━━━━━━━━━━━━━━━━━━ 41s 149ms/step - age_out_accuracy: 0.0452 - age_out_loss: 5.2556 - gender_out_accuracy: 0.9085 - gender_out_loss: 0.2142 - loss: 6.2677 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 6.7388 - val_gender_out_accuracy: 0.8994 - val_gender_out_loss: 0.2369 - val_loss: 7.7875
Epoch 38/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - age_out_accuracy: 0.0469 - age_out_loss: 5.2312 - gender_out_accuracy: 0.9057 - gender_out_loss: 0.2174 - loss: 6.2578
Epoch 38: val_loss did not improve from 7.78754
278/278 ━━━━━━━━━━━━━━━━━━━━ 40s 144ms/step - age_out_accuracy: 0.0469 - age_out_loss: 5.2315 - gender_out_accuracy: 0.9057 - gender_out_loss: 0.2174 - loss: 6.2581 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 8.4472 - val_gender_out_accuracy: 0.8905 - val_gender_out_loss: 0.2438 - val_loss: 9.5127
Epoch 39/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - age_out_accuracy: 0.0490 - age_out_loss: 5.1572 - gender_out_accuracy: 0.9044 - gender_out_loss: 0.2198 -

278/278 ━━━━━━━━━━━━━━━━━━━━ 39s 139ms/step - age_out_accuracy: 0.0456 - age_out_loss: 4.8066 - gender_out_accuracy: 0.9131 - gender_out_loss: 0.1947 - loss: 5.9735 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 6.4252 - val_gender_out_accuracy: 0.8900 - val_gender_out_loss: 0.2520 - val_loss: 7.6564
Epoch 63/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - age_out_accuracy: 0.0478 - age_out_loss: 4.8834 - gender_out_accuracy: 0.9240 - gender_out_loss: 0.1768 - loss: 6.0386
Epoch 63: val_loss did not improve from 7.65640
278/278 ━━━━━━━━━━━━━━━━━━━━ 38s 138ms/step - age_out_accuracy: 0.0478 - age_out_loss: 4.8832 - gender_out_accuracy: 0.9240 - gender_out_loss: 0.1769 - loss: 6.0385 - val_age_out_accuracy: 0.0474 - val_age_out_loss: 6.6055 - val_gender_out_accuracy: 0.8903 - val_gender_out_loss: 0.2629 - val_loss: 7.8534
Epoch 64/250
278/278 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - age_out_accuracy: 0.0483 - age_out_loss: 4.7407 - gender_out_accuracy: 0.9212 - gender_out_loss: 0.1867 -